1 - get the nodes state observations : their informations through the rounds + the accuracy of their models

In [1]:
import json
# Load the JSON data from the file scratch/subdir/results/score_alpha02_nodes_150/blockchain.json
# with open('../results/score_alpha02_nodes_150/blockchain.json', 'r') as f:
#     data = json.load(f)
# with open('../results/score_alpha05_nodes_150/blockchain.json', 'r') as f:
#     data = json.load(f)
# with open('../results/score_alpha08_nodes150/blockchain.json', 'r') as f:
#     data = json.load(f)
# with open('../results/score_dropout10/blockchain.json', 'r') as f:
#     data = json.load(f)
# with open('../results/score_dropout30/blockchain.json', 'r') as f:
#     data = json.load(f)
# with open('../results/score_dropout60/blockchain.json', 'r') as f:
#     data = json.load(f)
# with open('../results/score_malicious10/blockchain.json', 'r') as f:
#     data = json.load(f)
# with open('../results/score_malicious30/blockchain.json', 'r') as f:
#     data = json.load(f)
with open('../results/second_try/score/blockchain.json', 'r') as f:
    data = json.load(f)

# Create a list to store node information for each block
node_info_by_block = {}
transitions = []
nodes_by_block=[]
# transition = {"obs" : , "obs_" , "action" : , "reward" : ,"done" : }
# Iterate through the transactions of all blocks
for json_obj in data:
    block_id = json_obj['BlockId']

    for transaction in json_obj['Transactions']:
        content = transaction.get('Content', {})
        message_type = content.get('message_type')
        node_id = content.get('nodeId')

        if message_type == 1 and node_id is not None:
            availability = int(content.get('availability'))
            honesty = content.get('honesty')
            dataset_size = content.get('datasetSize')
            freq = content.get('freq')
            trans_rate = content.get('transRate')
            task = content.get('task')
            dropout = content.get('dropout', 0.0)

            # Find accuracy for the corresponding node in message_type 2 transactions
            accuracy = 0.0
            for json_obj2 in data:
                if json_obj2['BlockId'] == block_id:
                    for transaction2 in json_obj2['Transactions']:
                        if transaction2['Content'].get('message_type') == 2 and transaction2['Content'].get('nodeId') == node_id and transaction2['Content'].get('type') == 0:
                            accuracy = transaction2['Content'].get('accuracy', 0.0)
                            break
                    if accuracy != 0.0:
                        break

            node_info = [node_id, availability, honesty, dataset_size, freq, trans_rate, dropout, accuracy]

            if block_id not in node_info_by_block:
                node_info_by_block[block_id] = []

            node_info_by_block[block_id].append(node_info)

# Sort the node lists within each block by nodeId
for nodes in node_info_by_block.values():
    nodes.sort(key=lambda x: x[0])  # Sort based on nodeId

# Print or use node_info_by_block as needed
for block_id, nodes in node_info_by_block.items():
    print(f"Block {block_id}:")
    nodes_by_block.append(nodes)
    for node_info in nodes:
        print(node_info)
   


Block 1/0:
[0, 1, 0.0, 250, 180.0, 530.0, 0.0, 50.0]
[1, 1, 0.0, 500, 170.0, 950.0, 0.0, 59.0]
[2, 1, 0.0, 220, 160.0, 330.0, 0.0, 36.36]
[3, 1, 0.0, 180, 280.0, 860.0, 0.0, 0.0]
[4, 1, 0.0, 360, 230.0, 360.0, 0.0, 54.17]
[5, 1, 0.0, 390, 190.0, 590.0, 0.0, 48.72]
[6, 1, 0.0, 90, 260.0, 860.0, 1.0, 0.0]
[7, 1, 0.0, 240, 200.0, 210.0, 1.0, 0.0]
[8, 1, 0.0, 190, 120.0, 230.0, 0.0, 0.0]
[9, 1, 0.0, 200, 130.0, 540.0, 0.0, 42.5]
[10, 0, 0.0, 380, 220.0, 870.0, 0.0, 0.0]
[11, 1, 0.0, 90, 120.0, 340.0, 0.0, 27.78]
[12, 1, 0.0, 230, 100.0, 630.0, 1.0, 0.0]
[13, 0, 0.0, 290, 70.0, 570.0, 0.0, 0.0]
[14, 1, 0.0, 430, 280.0, 280.0, 1.0, 0.0]
[15, 1, 0.0, 410, 220.0, 840.0, 0.0, 0.0]
[16, 0, 0.0, 140, 130.0, 190.0, 0.0, 0.0]
[17, 0, 0.0, 440, 260.0, 390.0, 1.0, 0.0]
[18, 1, 0.0, 230, 110.0, 940.0, 0.0, 0.0]
[19, 0, 0.0, 100, 190.0, 660.0, 0.0, 0.0]
[20, 1, 0.0, 170, 260.0, 540.0, 0.0, 0.0]
[21, 0, 0.0, 270, 70.0, 160.0, 0.0, 0.0]
[22, 1, 0.0, 420, 160.0, 820.0, 1.0, 0.0]
[23, 1, 0.0, 250, 120.0, 8

2 - get the global accuracy for each round

In [2]:
# Create a list to store the global accuracy values
global_accuracy = []

# Iterate through the transactions of all blocks
for json_obj in data:
    block_id = json_obj['BlockId']
    
    # Find the transaction with message_type equal to 2 and type equal to 2
    accuracy_transaction = None
    for transaction in json_obj['Transactions']:
        content = transaction['Content']
        if content.get('message_type') == 2 and content.get('type') == 2:
            accuracy_transaction = transaction
            break
    
    if accuracy_transaction is not None:
        accuracy = accuracy_transaction['Content'].get('accuracy', 0.0)
        global_accuracy.append(accuracy)

# Print or use global_accuracy as needed
print("Global Accuracy Values:", global_accuracy)

Global Accuracy Values: [47.78, 62.22, 76.0, 76.0, 78.0, 83.41, 81.67, 85.0, 83.33, 86.11, 84.44, 86.67, 87.22, 86.67, 86.67, 87.22, 86.0, 86.4, 87.6, 86.0, 85.6, 85.6, 85.6, 85.6, 87.35, 90.29, 89.71, 87.6, 90.73]


In [3]:
# Create a list to store the concatenated lists for each block
selected_nodes = []

# Iterate through the transactions of all blocks
for json_obj in data:
    block_id = json_obj['BlockId']
    
    # Find the transaction with message_type equal to 4
    aggregator_transaction = None
    for transaction in json_obj['Transactions']:
        if transaction['Content'].get('message_type') == 4:
            aggregator_transaction = transaction
            break
    
    if aggregator_transaction is not None:
        aggregators = aggregator_transaction['Content'].get('aggregators', [])
        trainers = aggregator_transaction['Content'].get('trainers', [])
        
        selected_nodes.append(aggregators + trainers)

# Print or use selected_nodes as needed
for block_id, selected_node_list in zip([json_obj['BlockId'] for json_obj in data], selected_nodes):
    print(f"Block {block_id}: {selected_node_list}")

Block 1/0: [6, 3, 46, 33, 27, 20, 32, 15, 34, 18, 37, 40, 23, 0, 31, 36, 9, 22, 1, 5, 12, 45, 48, 11, 4, 2, 41, 38, 14, 39]
Block 1/1: [3, 33, 27, 20, 15, 34, 18, 19, 23, 0, 9, 10, 30, 1, 5, 11, 4, 2, 41, 38, 39, 13, 35, 29, 47, 43, 8, 7, 25, 16]
Block 1/2: [33, 20, 27, 15, 34, 3, 46, 19, 23, 0, 9, 10, 30, 1, 40, 11, 4, 44, 2, 49, 41, 38, 39, 17, 13, 35, 29, 47, 43, 16]
Block 1/3: [33, 15, 27, 20, 34, 46, 23, 19, 0, 18, 3, 9, 10, 30, 1, 40, 11, 5, 4, 44, 2, 41, 38, 39, 29, 47, 43, 16, 8, 26]
Block 1/4: [33, 15, 20, 34, 27, 46, 23, 0, 3, 9, 10, 30, 1, 19, 11, 5, 4, 44, 2, 41, 38, 39, 49, 29, 13, 47, 43, 35, 16, 8]
Block 1/5: [15, 27, 20, 34, 33, 46, 23, 0, 18, 9, 10, 30, 1, 19, 11, 5, 40, 4, 44, 2, 3, 41, 38, 49, 29, 13, 47, 43, 35, 16]
Block 1/6: [27, 33, 34, 23, 0, 20, 18, 9, 10, 30, 1, 19, 5, 40, 4, 44, 2, 3, 41, 38, 49, 29, 13, 47, 43, 35, 16, 8, 26, 6]
Block 1/7: [27, 33, 34, 23, 0, 20, 15, 46, 9, 10, 30, 1, 19, 18, 5, 40, 4, 44, 2, 3, 41, 38, 49, 29, 13, 47, 39, 43, 35, 16]
Block 

In [4]:
round_reward = []
transitions = []

for block_id in range(len(node_info_by_block) - 1):  # Fixed the range
    # print(len(node_info_by_block) - 1)
    # print(f"Block {block_id}:")
    nodes_rewards = []
    
    for node_index in selected_nodes[block_id]:
        # print("Node index: ", node_index)
        node_data = nodes_by_block[block_id][node_index]
        reward = node_data[-1]
        # print("Original reward: ", reward)
        
        if reward != 0.0:
            reward *= node_data[2]
        else :
            reward = node_data[2]
        # print("Modified reward: ", reward)
        
        nodes_rewards.append(reward)
    
    # print("Nodes rewards:", nodes_rewards)
    cumulated_reward = sum(nodes_rewards)
    round_reward.append(cumulated_reward)
    
    transition = {
        "obs": nodes_by_block[block_id],
        "obs_": nodes_by_block[block_id + 1],
        "action": selected_nodes[block_id],
        "reward": cumulated_reward,
        "done": block_id == len(node_info_by_block) - 1  # Corrected the condition
    }
    transitions.append(transition)

for transition in transitions : 
    print(transition)


{'obs': [[0, 1, 0.0, 250, 180.0, 530.0, 0.0, 50.0], [1, 1, 0.0, 500, 170.0, 950.0, 0.0, 59.0], [2, 1, 0.0, 220, 160.0, 330.0, 0.0, 36.36], [3, 1, 0.0, 180, 280.0, 860.0, 0.0, 0.0], [4, 1, 0.0, 360, 230.0, 360.0, 0.0, 54.17], [5, 1, 0.0, 390, 190.0, 590.0, 0.0, 48.72], [6, 1, 0.0, 90, 260.0, 860.0, 1.0, 0.0], [7, 1, 0.0, 240, 200.0, 210.0, 1.0, 0.0], [8, 1, 0.0, 190, 120.0, 230.0, 0.0, 0.0], [9, 1, 0.0, 200, 130.0, 540.0, 0.0, 42.5], [10, 0, 0.0, 380, 220.0, 870.0, 0.0, 0.0], [11, 1, 0.0, 90, 120.0, 340.0, 0.0, 27.78], [12, 1, 0.0, 230, 100.0, 630.0, 1.0, 0.0], [13, 0, 0.0, 290, 70.0, 570.0, 0.0, 0.0], [14, 1, 0.0, 430, 280.0, 280.0, 1.0, 0.0], [15, 1, 0.0, 410, 220.0, 840.0, 0.0, 0.0], [16, 0, 0.0, 140, 130.0, 190.0, 0.0, 0.0], [17, 0, 0.0, 440, 260.0, 390.0, 1.0, 0.0], [18, 1, 0.0, 230, 110.0, 940.0, 0.0, 0.0], [19, 0, 0.0, 100, 190.0, 660.0, 0.0, 0.0], [20, 1, 0.0, 170, 260.0, 540.0, 0.0, 0.0], [21, 0, 0.0, 270, 70.0, 160.0, 0.0, 0.0], [22, 1, 0.0, 420, 160.0, 820.0, 1.0, 0.0], [23, 

In [9]:
import sys
sys.path.insert(0, '../')  # Add the directory containing "DRL" to the path

from agent import node_selection_agent as ag
from environment import node_selection_env as nds

total_nodes = 50
select_trainer = 20
select_Agg = 10
num_features = 8
target_accuracy = 99
max_rounds= 30

#create the nodes selection environment accoding to the observation
envNodeSelect = nds.FLNodeSelectionEnv(total_nodes,select_Agg,select_trainer, num_features=num_features, target=target_accuracy, max_rounds=max_rounds)
obs_shape = envNodeSelect.observation_space.sample()["current_state"].shape
print(obs_shape)
agent = ag.Agent(input_shape=obs_shape ,n_actions=obs_shape[0], env=envNodeSelect,max_actions=select_trainer+select_Agg) 
agent.load_models()

# train the agent with the transitions store the trasitions then make the agent learn
for transition in transitions : 
    agent.remember(transition["obs"],transition["action"], transition["reward"], transition["obs_"], transition["done"])
    agent.learn()

agent.save_models()


(50, 8)
.... loading models ....
...loading checkpoint...
...loading checkpoint...
...loading checkpoint...
...loading checkpoint...
...loading checkpoint...
.... finished loading models ....
.... saving models ....
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
updated the networks
.... saving models ....
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
updated the networks
.... saving models ....
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
updated the networks
.... saving models ....
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
updated the networks
.... saving models ....
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
upda

In [ ]:
from collections import defaultdict


# Create a dictionary to store the concatenated lists for each block
selected_nodes = defaultdict(list)

# Iterate through the transactions of all blocks
for json_obj in data:
    block_id = json_obj['BlockId']
    
    # Find the transaction with message_type equal to 4
    aggregator_transaction = None
    for transaction in json_obj['Transactions']:
        if transaction['Content'].get('message_type') == 4:
            aggregator_transaction = transaction
            break
    
    if aggregator_transaction is not None:
        aggregators = aggregator_transaction['Content'].get('aggregators', [])
        trainers = aggregator_transaction['Content'].get('trainers', [])
        
        selected_nodes[block_id] = aggregators + trainers

# Count how many times each node was selected
node_selection_counts = defaultdict(int)
for selected_node_list in selected_nodes.values():
    for node_id in selected_node_list:
        node_selection_counts[node_id] += 1

# Print or use node_selection_counts as needed
for node_id, count in node_selection_counts.items():
    print(f"Node {node_id} was selected {count} times.")


Node 121 was selected 25 times.
Node 14 was selected 3 times.
Node 145 was selected 27 times.
Node 49 was selected 10 times.
Node 125 was selected 26 times.
Node 105 was selected 27 times.
Node 15 was selected 9 times.
Node 43 was selected 10 times.
Node 108 was selected 23 times.
Node 13 was selected 6 times.
Node 127 was selected 23 times.
Node 57 was selected 7 times.
Node 91 was selected 9 times.
Node 12 was selected 8 times.
Node 65 was selected 9 times.
Node 30 was selected 5 times.
Node 77 was selected 8 times.
Node 32 was selected 3 times.
Node 86 was selected 11 times.
Node 119 was selected 25 times.
Node 42 was selected 7 times.
Node 20 was selected 6 times.
Node 123 was selected 29 times.
Node 95 was selected 3 times.
Node 39 was selected 5 times.
Node 28 was selected 6 times.
Node 16 was selected 5 times.
Node 87 was selected 8 times.
Node 134 was selected 26 times.
Node 106 was selected 25 times.
Node 97 was selected 5 times.
Node 47 was selected 9 times.
Node 73 was selec